In [34]:
import pandas as pd
import dash
from dash import html, dcc
from dash import no_update
from dash.dependencies import Input, Output
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
df = pd.read_csv( "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [36]:
max_playload= df['Payload Mass (kg)'].max()
num_keys=10
step=max_playload//(num_keys-1)



In [37]:
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions=True
app.layout = html.Div(children =[html.H1('SpaceX Launch Records Dashboard',
                                         style={'textAlign': 'center', 'color': 'white',
                                                'font-size':24}),
                                html.Div([
                                    html.Div([
                                        dcc.Dropdown(id='dropdown-chose',
                                                    options=[
                                                        {'label':'CCAFS LC-40', 'value' : 'CCAFS LC-40'},
                                                        {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                        {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                        {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                        {'label':'all site','value':'all'}
                                                    ],
                                                    placeholder='select a report type',
                                                    value='VAFB SLC-4E'),
                                    ],style={'width':'80%','padding':'3px','fontSize':'20px','textAlignLast':'center'}),
                                    html.Div([
                                        dcc.RangeSlider(id='range-chose',
                                                        min=0, max=df['Payload Mass (kg)'].max(), step=step,
                                                        marks={i * step: f'label {str(i * step)}' for i in range(num_keys)},
                                                        value=[0,df['Payload Mass (kg)'].max()])
                                    ],style={'width':'80%','padding':'3px','fontSize':'20px','textAlignLast':'center','color': 'white'})
                                    ]),
                                html.Div([
                                    html.Div(id='output-spacex', className='chart-grid', style={'display':'flex'}),
                                    html.Div(id='output-payload',className='chart-grid',style={'display':'flex'}),
                                ])    
                                ])


In [38]:
@app.callback(
    Output(component_id='output-spacex', component_property='children'),
    Input(component_id='dropdown-chose',component_property='value'))
def update_output_spacex(launch_site):
    if launch_site == 'all':
        df_all = df[df['class'] == 1]
        df_all = df_all.groupby(['Launch Site'])['class'].count().reset_index()
        Y_chart = dcc.Graph(
        figure = px.pie(df_all,values='class',names='Launch Site',title='barplot'))
    else:
        df_site = df[df['Launch Site'] == launch_site]['class'].value_counts().reset_index()
        Y_chart = dcc.Graph(
        figure = px.pie(df_site,values=df_site['count'],names=df_site['class'],title='barplot'))
    return Y_chart  

In [39]:
@app.callback(
    Output(component_id='output-payload', component_property='children'),
    Input(component_id='dropdown-chose',component_property='value'),
    Input(component_id='range-chose',component_property='value')
)
def update_output_payload(launch_site,payload):
    if launch_site == 'all':
        df_payload = df[(df['Payload Mass (kg)'] >= payload[0]) & (df['Payload Mass (kg)'] <= payload[1]) ]
        Y_chart=dcc.Graph(
            figure= px.scatter(df_payload,x='Payload Mass (kg)',y='Launch Site',title='scatter plot',color="Booster Version Category")
        )
    else:
        df_payload = df[(df['Launch Site'] == launch_site) & (df['Payload Mass (kg)'] >= payload[0]) & (df['Payload Mass (kg)'] <= payload[1])]
        Y_chart=dcc.Graph(
        figure= px.scatter(df_payload,x='Payload Mass (kg)',y='Launch Site',title='scatter plot',color="Booster Version Category")
        )
    return Y_chart

In [40]:
if __name__ == '__main__':
    app.run_server()